In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [2]:
from torchvision import transforms
test_transform = transforms.Compose([
                                     transforms.Grayscale(1),
                                     #transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     #transforms.Normalize([0.485, ], [0.229, ])
                                     ])


In [3]:
df = pd.read_csv('C:/Users/liFangzheng/Desktop/LI Peiyue/umap/umap_data/umap_profile.csv')


In [5]:
model = torch.load('C:/Users/liFangzheng/Desktop/LI Peiyue/save/best-0.964.pth')
model = model.eval().to(device)

In [6]:
from torchvision.models.feature_extraction import create_feature_extractor

In [7]:
model_trunc = create_feature_extractor(model, return_nodes={'avgpool': 'semantic_feature'})

In [8]:

encoding_array = []
img_path_list = []

for img_path in tqdm(df['path']):
    img_path_list.append(img_path)
    img_pil = Image.open(img_path)
    input_img = test_transform(img_pil).unsqueeze(0).to(device) 
    feature = model_trunc(input_img)['semantic_feature'].squeeze().detach().cpu().numpy() 
    encoding_array.append(feature)
encoding_array = np.array(encoding_array)

100%|███████████████████████████████████████████████████████████████████████████████| 664/664 [00:06<00:00, 101.44it/s]


In [9]:
encoding_array.shape

(664, 512)

In [10]:
encoding_array

array([[0.8174653 , 0.28499305, 1.1201572 , ..., 0.6347453 , 1.6766148 ,
        1.065004  ],
       [1.6769117 , 0.74230444, 1.7574161 , ..., 0.65568167, 2.0572786 ,
        1.9267365 ],
       [1.4911529 , 0.7535913 , 0.80893576, ..., 1.2474791 , 1.5025234 ,
        1.7553607 ],
       ...,
       [1.5931678 , 2.1213508 , 2.5299962 , ..., 0.04556078, 0.23655722,
        1.1093171 ],
       [2.8854694 , 3.2155254 , 2.3882499 , ..., 0.13452935, 0.14327994,
        2.115413  ],
       [2.9337232 , 3.6421068 , 2.8412268 , ..., 0.10173308, 0.41182646,
        1.880815  ]], dtype=float32)

In [11]:
np.save('C:/Users/liFangzheng/Desktop/LI Peiyue/umap/umap_data/test_feature.npy', encoding_array)